In [1]:
import mlflow
print(mlflow.__version__)

2.12.1


In [2]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("check-localhost-connection")

2024/05/03 17:04:13 INFO mlflow.tracking.fluent: Experiment with name 'check-localhost-connection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/272185727337227628', creation_time=1714770253267, experiment_id='272185727337227628', last_update_time=1714770253267, lifecycle_stage='active', name='check-localhost-connection', tags={}>

In [3]:
with mlflow.start_run():
    mlflow.log_metric("foo", 1)
    mlflow.log_metric("bar", 2)